In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from sklearn import mixture
from sklearn.utils import shuffle
from skimage import measure
from glob import glob
import os
from multiprocessing import Pool, cpu_count
from functools import partial
from subprocess import check_output
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import Model
K.set_image_dim_ordering('tf')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train_crop_224.npy')
train_target = np.load('train_crop_target_224.npy')
test_data = np.load('test_stg2_crop_224.npy')
test_id = np.load('test_stg2_crop_id_224.npy')

Using TensorFlow backend.


In [2]:
train_data

array([[[[ 0.01568628,  0.01176471,  0.01568628, ...,  0.00392157,
           0.00392157,  0.        ],
         [ 0.01176471,  0.01176471,  0.01176471, ...,  0.00392157,
           0.00392157,  0.00392157],
         [ 0.01176471,  0.01176471,  0.01176471, ...,  0.00392157,
           0.00392157,  0.00392157],
         ..., 
         [ 0.        ,  0.00392157,  0.00392157, ...,  0.00784314,
           0.00784314,  0.00784314],
         [ 0.00392157,  0.00392157,  0.00392157, ...,  0.00784314,
           0.00784314,  0.00392157],
         [ 0.00392157,  0.        ,  0.        , ...,  0.00392157,
           0.00392157,  0.00392157]],

        [[ 0.01568628,  0.01176471,  0.01568628, ...,  0.00392157,
           0.00392157,  0.        ],
         [ 0.01176471,  0.01176471,  0.01176471, ...,  0.00392157,
           0.00392157,  0.00392157],
         [ 0.01176471,  0.01176471,  0.01176471, ...,  0.00392157,
           0.00392157,  0.00392157],
         ..., 
         [ 0.        ,  0.003921

In [2]:
# convert (3, 64, 64)-> (64, 64, 3)

train_data = train_data.transpose((0, 2, 3, 1))
test_data = test_data.transpose((0, 2, 3, 1))
# avoid memory over flow if the shape is 224 * 224
# the fllowing is gonna be memory error... if test_size is large
# train_data, _ , train_target, _ = train_test_split(train_data, train_target,test_size=0.8, random_state=17)
# train_target = train_target

In [4]:
# train_data = np.load('train_crop_64.npy')
train_data.shape

(8205, 224, 224, 3)

In [5]:
tes = pd.DataFrame({'target': train_target})

In [6]:
tes['target'].value_counts()

1    4344
2    2426
0    1435
Name: target, dtype: int64

In [7]:
train_target.shape

(8205,)

In [8]:
input_tensor = Input(dtype='float32', shape=(224,224, 3))
vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(224, 224, 3))
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
#set 3 for three types clf
top_model.add(Dense(3, activation='sigmoid'))

In [10]:
model = Model(input=vgg16_model.input, output=top_model(vgg16_model.output))

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  if __name__ == '__main__':


In [11]:
print('vgg16_model: ', vgg16_model)
print('top_model: ', top_model)
print('model: ', model)

vgg16_model:  <keras.engine.training.Model object at 0x7fd0c5280438>
top_model:  <keras.models.Sequential object at 0x7fd0c520fc50>
model:  <keras.engine.training.Model object at 0x7fd39e65dac8>


In [12]:
#display layers
for i in range(len(model.layers)):
    print(i, model.layers[i])

0 <keras.engine.topology.InputLayer object at 0x7fd39e65dda0>
1 <keras.layers.convolutional.Conv2D object at 0x7fd39e65dd68>
2 <keras.layers.convolutional.Conv2D object at 0x7fd39e65a0f0>
3 <keras.layers.pooling.MaxPooling2D object at 0x7fd0c53a2f28>
4 <keras.layers.convolutional.Conv2D object at 0x7fd0c5361f98>
5 <keras.layers.convolutional.Conv2D object at 0x7fd0c5370c18>
6 <keras.layers.pooling.MaxPooling2D object at 0x7fd0c530acf8>
7 <keras.layers.convolutional.Conv2D object at 0x7fd0c5318828>
8 <keras.layers.convolutional.Conv2D object at 0x7fd0c5325eb8>
9 <keras.layers.convolutional.Conv2D object at 0x7fd0c5333588>
10 <keras.layers.pooling.MaxPooling2D object at 0x7fd0c52cfc18>
11 <keras.layers.convolutional.Conv2D object at 0x7fd0c52eb198>
12 <keras.layers.convolutional.Conv2D object at 0x7fd0c52fa828>
13 <keras.layers.convolutional.Conv2D object at 0x7fd0c52869e8>
14 <keras.layers.pooling.MaxPooling2D object at 0x7fd0c52a0f98>
15 <keras.layers.convolutional.Conv2D object at 0x7

In [13]:
#freeze layers
for layer in model.layers[:15]:
    layer.trainable = False

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

In [5]:
# datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
datagen = ImageDataGenerator()
datagen.fit(train_data)

x_train, x_val_train, y_train, y_val_train = train_test_split(train_data, train_target,test_size=0.4, random_state=17)

In [16]:
# x_train = preprocess_input(x_train)

In [17]:
# x_train *= 255

In [18]:
# x_train

In [19]:
y_val_train.shape

(3282,)

In [20]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), 
# history = model.fit_generator(datagen.flow(train_data, train_target, batch_size=15, shuffle=True), 
                    nb_epoch=30,  samples_per_epoch=len(x_train), verbose=1,
                    validation_data=(x_val_train, y_val_train))

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, epochs=30, steps_per_epoch=328, validation_data=(array([[[...)`


Epoch 1/30
328/328 [==============================] - 1063s - loss: 1.0162 - acc: 0.5203 - val_loss: 1.0067 - val_acc: 0.5393
Epoch 2/30
328/328 [==============================] - 1063s - loss: 0.9970 - acc: 0.5242 - val_loss: 0.9656 - val_acc: 0.5393
Epoch 3/30
328/328 [==============================] - 1062s - loss: 0.9681 - acc: 0.5252 - val_loss: 0.9527 - val_acc: 0.5524
Epoch 4/30
328/328 [==============================] - 1061s - loss: 0.9541 - acc: 0.5285 - val_loss: 0.9444 - val_acc: 0.5536
Epoch 5/30
328/328 [==============================] - 1064s - loss: 0.9324 - acc: 0.5388 - val_loss: 0.9246 - val_acc: 0.5576
Epoch 6/30
328/328 [==============================] - 1063s - loss: 0.9202 - acc: 0.5402 - val_loss: 0.9169 - val_acc: 0.5634
Epoch 7/30
328/328 [==============================] - 1063s - loss: 0.8967 - acc: 0.5500 - val_loss: 0.9143 - val_acc: 0.5615
Epoch 8/30
328/328 [==============================] - 1062s - loss: 0.8839 - acc: 0.5533 - val_loss: 0.8985 - val_acc:

KeyboardInterrupt: 

In [21]:
model.save('vgg16_224_21epoc')

In [ ]:
score = model.evaluate(x_val_train, y_val_train)
score

In [ ]:
plt.plot(history.history['val_loss'])

In [ ]:
# test_data = np.load('test_crop_64.npy')
# test_id = np.load('test_crop_id_64.npy')
# test_data = test_data.transpose((0, 2,3,1))
predictions = model.predict(test_data, verbose=0)

In [ ]:
predictions

In [ ]:
df = pd.DataFrame(predictions, columns=['Type_1', 'Type_2', 'Type_3'])
df['image_name'] = test_id
df.to_csv('2stg_submission_4.csv', index=False)

# load model

In [3]:
from keras.models import load_model
model224_21epoch = load_model('vgg16_224_21epoc')

In [4]:
model224_21epoch.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

In [ ]:
history_2 = model224_21epoch.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), 
                    nb_epoch=10,  samples_per_epoch=len(x_train), verbose=1, 
                    validation_data=(x_val_train, y_val_train))

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, steps_per_epoch=328, validation_data=(array([[[..., epochs=10)`
  app.launch_new_instance()


Epoch 1/10
 57/328 [====>.........................] - ETA: 566s - loss: 0.4361 - acc: 0.8257

In [ ]:
model224_21epoch.save('vgg16_224_31epoc')

In [ ]:
score = model_20epoch.evaluate(x_val_train, y_val_train)
score

In [ ]:
plt.plot(history_2.history['val_loss'])